Now that we have a few classes and funcs created, let's look at what could go wrong.

In [1]:
from arr import ContractHeader, annualize
from datetime import date

### Leap Years

Start with a 20k TCV contract starting on January 2024 and an ending on January 2026.

In [2]:
contract = ContractHeader(20_000, date(2024, 1, 15), date(2026, 1, 14))

dont_generalize_leap_year = annualize(contract, date(2024, 6, 30), "Day", False)
generalize_leap_year = annualize(contract, date(2024, 6, 30), "Day", True)

print("Results of leap year:")
print(f"Not generalizing leap year: {dont_generalize_leap_year:.2f}")
print(f"Generalizing leap year: {generalize_leap_year:.2f}")

Results of leap year:
Not generalizing leap year: 10013.68
Generalizing leap year: 10000.00


Simple thought would suggest that 20k TCV of 2 years will have a 10k ACV, but **take caution when getting granular as days, because leap years will make an appearance**. Let's look at the implication of this over time. Consider the below 3 year deal and let's ask, "What is the ACV of this contract month over month? and how do leap years show in the data?"

$
\begin{matrix}
\text{Contract} & \text{Start Date} & \text{End Date} & \text{Total Contract Value} \\
\text{001} & \text{06/15/2023} & \text{06/14/2025} & 30000 
\end{matrix}
$

In [20]:
from arr import get_end_of_month_range
import numpy as np
import plotly.express as px


contract = ContractHeader(30_000, date(2023, 6, 15), date(2025, 6, 14))

date_range = get_end_of_month_range(date(2023, 6, 1), date(2025, 6, 1))

results_w_leap_year = np.array(
    [annualize(contract, month, "Day", False) for month in date_range]
)

results_wo_leap_year = np.array(
    [annualize(contract, month, "Day", True) for month in date_range]
)
fig1 = px.line(x=date_range, y=results_w_leap_year, title="ACV w/ Leap Year")
fig1.show()

fig2 = px.line(x=date_range, y=results_wo_leap_year, title="ACV w/o Leap Year")
fig2.show()

<div class="alert alert-block alert-warning">
The difference here is only $20 dollars, but imagine this scaled across an enterprise company. Contracts with terms over leap years will see their ARR rise in the beginning of a leap year and fall at the end of a leap year. My recommendation, if you are going to annualize by day, <a href="https://en.wikipedia.org/wiki/KISS_principle">K.I.S.S</a>. <b>Hard code the day value to be 365 and move on.</b> I can see some reasons where you want to see this swing in data, but many companies will be just fine with 365 days every year.
</div>

Let's continue the leap year analysis, with another contract.

In [4]:
short_contract = ContractHeader(25_000, date(2024, 1, 1), date(2024, 3, 31))

no_gen_leap_year = annualize(
    short_contract, date(2024, 3, 31), "Day", False, print_details=True
)
print(f"\nNot generalizing leap year: {no_gen_leap_year:.2f}\n")

print("-" * 55)

gen_leap_year = annualize(
    short_contract, date(2024, 3, 31), "Day", True, print_details=True
)
print(f"\nGeneralizing leap year: {gen_leap_year:.2f}\n")

ContractHeader(amount=25000, start_date=datetime.date(2024, 1, 1), end_date=datetime.date(2024, 3, 31), booking_date=datetime.date(2024, 1, 1))
Period: 2024-03-31
Generalize Leap Year: False
Time Interval: Day - 366
Contract Term: 91

Not generalizing leap year: 100549.45

-------------------------------------------------------
ContractHeader(amount=25000, start_date=datetime.date(2024, 1, 1), end_date=datetime.date(2024, 3, 31), booking_date=datetime.date(2024, 1, 1))
Period: 2024-03-31
Generalize Leap Year: True
Time Interval: Day - 365
Contract Term: 90

Generalizing leap year: 101388.89



Why is this 25k contract still showing an odd amount when annualizing by day, even with `generalize_leap_year` set to `True`? Let's do some math.

$\begin{matrix}
\text{Formula:} &
\text{Annual Contract Value} = \text{Total Contract Value} * (\frac{\text{Time Interval (Year)}}{\text{Contract Term Length}})\\
\\
\text{Generalizing leap year = False:} &
100,549 = 25,000 * (\frac{366}{91})\\
\\
\text{Generalizing leap year = True:} &
101,388 = 25,000 * (\frac{365}{90})
\end{matrix}
$

General wisdom tells us that this 3 month contract should be multiplied by 4. 12 months in a year divided by 3 months of the contract multiplied by total contract value. $25,000 * \frac{12}{3} = 100,000$. But regardless of removing leap days from the equation, you will always have less days in the first half of a year than in the second half of the year. Which means, annualizing by day in the first half will always give an unconventional result when comparing to the full year.

<div class="alert alert-block alert-warning">
You've been warned. Another risk of annualizing by day. Still do-able, but leap years add a ton of implications.
</div>

### Contracts Less Than 1 Year

Sales people [*always be closing*](https://www.investopedia.com/terms/a/always-be-closing.asp). Which means, never underestimate the creativity of a sales team. They will do what they need to do to make the company money. Which means contracts could have shorter lengths, ramp, and could have all kinds of discounts or variability. For this section, the focus will be on shorter contracts. Shorter contracts intended to renew, can force a big swing in the reporting when not renewed.

First, I'll setup some code to show this smaller contract in the context of other contracts. I'll also introduce two new functions.

In [5]:
from arr import Contract, ContractLine

short_contract = Contract(
    1,
    ContractHeader(10_000, date(2024, 1, 1), date(2024, 3, 31)),
    [
        ContractLine(2_500, date(2024, 1, 1), date(2024, 3, 31), 1, True),
        ContractLine(2_500, date(2024, 1, 1), date(2024, 3, 31), 2, True),
        ContractLine(2_500, date(2024, 1, 1), date(2024, 3, 31), 3, True),
        ContractLine(2_500, date(2024, 1, 1), date(2024, 3, 31), 4, True),
    ],
)

reg_contract1 = Contract(
    2,
    ContractHeader(10_000, date(2024, 1, 1), date(2024, 12, 31)),
    [
        ContractLine(10_000, date(2024, 1, 1), date(2024, 12, 31), 1, True),
    ],
)

reg_contract2 = Contract(
    3,
    ContractHeader(10_000, date(2024, 1, 1), date(2024, 12, 31)),
    [
        ContractLine(10_000, date(2024, 1, 1), date(2024, 12, 31), 1, True),
    ],
)

print(short_contract, reg_contract1, reg_contract2, sep=f"\n{'-'*55}\n")

          Contract #1 - $10,000 - 2024-01-01           
                2024-01-01 - 2024-03-31                
*******************************************************
sku	start date	end date	amount	renewable
*******************************************************
1	2024-01-01	2024-03-31	2500	True
2	2024-01-01	2024-03-31	2500	True
3	2024-01-01	2024-03-31	2500	True
4	2024-01-01	2024-03-31	2500	True
*******************************************************

-------------------------------------------------------
          Contract #2 - $10,000 - 2024-01-01           
                2024-01-01 - 2024-12-31                
*******************************************************
sku	start date	end date	amount	renewable
*******************************************************
1	2024-01-01	2024-12-31	10000	True
*******************************************************

-------------------------------------------------------
          Contract #3 - $10,000 - 2024-01-01           
                2

In [6]:
from arr import build_contracts_table
from inspect import getsource
from IPython.display import Code

Code(getsource(build_contracts_table), language="python3")

def build_contracts_table(contracts: List[Contract]) -> pd.DataFrame:
    """Takes inputed Contract(s) converts to usable DataFrame.

    Args:
        contracts (List[Contract]): List of `Contract` dataclass.

    Returns:
        pd.DataFrame: DataFrame of Contract(s). 1 Row == 1 Line in a Contract.
            Both header and line information is in each row.
    """
    contracts = pd.DataFrame(contracts)
    headers = pd.DataFrame.from_records(contracts["header"])
    headers.rename(
        columns={
            "amount": "header_amount",
            "start_date": "header_start_date",
            "end_date": "header_end_date",
        },
        inplace=True,
    )
    lines_temp = contracts["lines"].explode()
    lines = pd.json_normalize(lines_temp)
    lines.index = lines_temp.index
    lines.rename(
        columns={
            "amount": "line_amount",
            "start_date": "line_start_date",
            "end_date": "line_end_date",
        },
        inplace=True,
    )
    df = (
        contracts.merge(headers, left_index=True, right_index=True)
        .merge(lines, left_index=True, right_index=True)
        .drop(columns=["header", "lines"])
        .reset_index(drop=True)
    )
    return df

In [7]:
build_contracts_table([short_contract, reg_contract1, reg_contract2])

,id,header_amount,header_start_date,header_end_date,booking_date,line_amount,line_start_date,line_end_date,item_sku,renewable
0,1,10000,2024-01-01,2024-03-31,2024-01-01,2500,2024-01-01,2024-03-31,1,True
1,1,10000,2024-01-01,2024-03-31,2024-01-01,2500,2024-01-01,2024-03-31,2,True
2,1,10000,2024-01-01,2024-03-31,2024-01-01,2500,2024-01-01,2024-03-31,3,True
3,1,10000,2024-01-01,2024-03-31,2024-01-01,2500,2024-01-01,2024-03-31,4,True
4,2,10000,2024-01-01,2024-12-31,2024-01-01,10000,2024-01-01,2024-12-31,1,True
5,3,10000,2024-01-01,2024-12-31,2024-01-01,10000,2024-01-01,2024-12-31,1,True


Take note of the $\text{2,500}$ line amounts for Contract # 1.

In [8]:
from arr import build_acv_table

Code(getsource(build_acv_table), language="python3")

def build_acv_table(
    contracts: List[Contract], by_lines: bool = True, include_deferred: bool = True
) -> pd.DataFrame:
    """Builds ACV DataFrame.

    Takes list of inputed `Contracts` and first builds a regular
    contract table with `build_contracts_table`. Then cross-joins
    with possible date values from `get_end_of_month_range`.
    Month range is determined by oldest and newest possible dates
    that are provided in the contracts. Depending on annualizing
    by header or by lines it will annualize the expected results.

    Args:
        contracts (List[Contract]): A list of the `Contract` dataclass.
        by_lines (bool, optional): bool to determine whether to
            annualize by header or by lines. If `True`, will annualize
            by lines, `False` will annualize by header. Defaults to True.
        include_deferred (bool, optional): bool to determine whether to
            include deferred ARR or not. If `True` data will include
            deferred ARR. `False` data will not include deferred ARR.
            Defaults to True.

    Returns:
        pd.DataFrame: Pandas dataframe as a matrix. The dates are the
            columns and contract id, item_sku, and renewability as
            the rows. ACV will be the values.
    """
    # 1
    all_lines = []

    for contract in contracts:
        all_lines += contract.lines

    min_booking_date = min([contract.header.booking_date for contract in contracts])
    min_start_date = min([line.start_date for line in all_lines])
    min_date = min(min_booking_date, min_start_date)
    max_date = max([line.end_date for line in all_lines])

    range = pd.DataFrame(get_end_of_month_range(min_date, max_date)).rename(
        columns={0: "period"}
    )

    # 2
    acv_table = range.merge(build_contracts_table(contracts), how="cross")

    # 3
    col_name = "line" if by_lines else "header"

    acv_table["acv"] = acv_table.apply(
        lambda row: row_annualization(row, col_name, include_deferred),
        axis=1,
    )

    # 4
    cols = ["period", "id", "item_sku", "renewable"]

    acv_table = acv_table[["period", "id", "item_sku", "renewable", "acv"]]

    acv_table = (
        pd.DataFrame(acv_table.groupby(cols).sum())
        .unstack(level=0)
        .drop(
            columns=[
                col for col in acv_table.columns if col not in cols and col != "acv"
            ]
        )
    )

    return acv_table["acv"]

In [9]:
build_acv_table([short_contract, reg_contract1, reg_contract2])

period                 2024-01-31  2024-02-29  2024-03-31  2024-04-30  \
id item_sku renewable                                                   
1  1        True          10000.0     10000.0     10000.0         0.0   
   2        True          10000.0     10000.0     10000.0         0.0   
   3        True          10000.0     10000.0     10000.0         0.0   
   4        True          10000.0     10000.0     10000.0         0.0   
2  1        True          10000.0     10000.0     10000.0     10000.0   
3  1        True          10000.0     10000.0     10000.0     10000.0   

period                 2024-05-31  2024-06-30  2024-07-31  2024-08-31  \
id item_sku renewable                                                   
1  1        True              0.0         0.0         0.0         0.0   
   2        True              0.0         0.0         0.0         0.0   
   3        True              0.0         0.0         0.0         0.0   
   4        True              0.0         0.0         0.0         0.0   
2  1        True          10000.0     10000.0     10000.0     10000.0   
3  1        True          10000.0     10000.0     10000.0     10000.0   

period                 2024-09-30  2024-10-31  2024-11-30  2024-12-31  
id item_sku renewable                                                  
1  1        True              0.0         0.0         0.0         0.0  
   2        True              0.0         0.0         0.0         0.0  
   3        True              0.0         0.0         0.0         0.0  
   4        True              0.0         0.0         0.0         0.0  
2  1        True          10000.0     10000.0     10000.0     10000.0  
3  1        True          10000.0     10000.0     10000.0     10000.0

When annualizing contracts **shorter** than 1 year, the ACV increases from the TCV. Annualizing contracts **longer** than 1 year, the ACV decreases from TCV. The implications of shorter contracts mean the company will chase after a larger ACV in a shorter amount of time. The example above shows that. Contract # 1 has 4 lines at 2.5k each. When annualizing, the 2.5k increases into 10k, but quickly falls to 0 when the contract is over.

In my experience, shorter contracts are often used as a back door *trial/demo*. It's a way to give a customer the product for a short period of time. This means the renewal implications are not always the same as a full year contract or more, so you're likely to see the big swing in shorter contracts on the down side.

<div class="alert alert-block alert-warning">
At scale, seeing the quick rise and fall of shorter contracts can often push a business decision to not annualize contracts shorter than a 1 year time frame.
</div>

### Ramp Contracts

A [ramp contract](https://dealhub.io/glossary/ramp-deal/) will have a change in price during the term of the contract. A common reason for ramp contracts is to lock the customer in for multiple years. The first year starts at price, then the subsequent years increase in price over time.

In [10]:
ramp_contract = Contract(
    1,
    ContractHeader(36_000, date(2024, 1, 1), date(2026, 12, 31)),
    [
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 1, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 2, True),
        ContractLine(6_000, date(2025, 1, 1), date(2025, 12, 31), 1, True),
        ContractLine(6_000, date(2025, 1, 1), date(2025, 12, 31), 2, True),
        ContractLine(8_000, date(2026, 1, 1), date(2026, 12, 31), 1, True),
        ContractLine(8_000, date(2026, 1, 1), date(2026, 12, 31), 2, True),
    ],
)
ramp_contract

          Contract #1 - $36,000 - 2024-01-01           
                2024-01-01 - 2026-12-31                
*******************************************************
sku	start date	end date	amount	renewable
*******************************************************
1	2024-01-01	2024-12-31	4000	True
2	2024-01-01	2024-12-31	4000	True
1	2025-01-01	2025-12-31	6000	True
2	2025-01-01	2025-12-31	6000	True
1	2026-01-01	2026-12-31	8000	True
2	2026-01-01	2026-12-31	8000	True
*******************************************************

Contract # 1 has a TCV of $\text{36k}$, over the course of 3 years. It has two different items in the contract and ramps up 2k each year for each item. Often times, ACV calculations will only annualize by header dates of contracts. Annualizing at the header does not give the full story.

In [11]:
by_lines = build_acv_table([ramp_contract])
by_header = build_acv_table([ramp_contract], False)

print("ARR of Ramp Contract by Header Dates")
display(by_header)
print("ARR of Ramp Contract by Line Dates")
display(by_lines)

ARR of Ramp Contract by Header Dates


period                 2024-01-31  2024-02-29  2024-03-31  2024-04-30  \
id item_sku renewable                                                   
1  1        True           6000.0      6000.0      6000.0      6000.0   
   2        True           6000.0      6000.0      6000.0      6000.0   

period                 2024-05-31  2024-06-30  2024-07-31  2024-08-31  \
id item_sku renewable                                                   
1  1        True           6000.0      6000.0      6000.0      6000.0   
   2        True           6000.0      6000.0      6000.0      6000.0   

period                 2024-09-30  2024-10-31  ...  2026-03-31  2026-04-30  \
id item_sku renewable                          ...                           
1  1        True           6000.0      6000.0  ...      6000.0      6000.0   
   2        True           6000.0      6000.0  ...      6000.0      6000.0   

period                 2026-05-31  2026-06-30  2026-07-31  2026-08-31  \
id item_sku renewable                                                   
1  1        True           6000.0      6000.0      6000.0      6000.0   
   2        True           6000.0      6000.0      6000.0      6000.0   

period                 2026-09-30  2026-10-31  2026-11-30  2026-12-31  
id item_sku renewable                                                  
1  1        True           6000.0      6000.0      6000.0      6000.0  
   2        True           6000.0      6000.0      6000.0      6000.0  

[2 rows x 36 columns]

ARR of Ramp Contract by Line Dates


period                 2024-01-31  2024-02-29  2024-03-31  2024-04-30  \
id item_sku renewable                                                   
1  1        True          18000.0     18000.0     18000.0     18000.0   
   2        True          18000.0     18000.0     18000.0     18000.0   

period                 2024-05-31  2024-06-30  2024-07-31  2024-08-31  \
id item_sku renewable                                                   
1  1        True          18000.0     18000.0     18000.0     18000.0   
   2        True          18000.0     18000.0     18000.0     18000.0   

period                 2024-09-30  2024-10-31  ...  2026-03-31  2026-04-30  \
id item_sku renewable                          ...                           
1  1        True          18000.0     18000.0  ...      8000.0      8000.0   
   2        True          18000.0     18000.0  ...      8000.0      8000.0   

period                 2026-05-31  2026-06-30  2026-07-31  2026-08-31  \
id item_sku renewable                                                   
1  1        True           8000.0      8000.0      8000.0      8000.0   
   2        True           8000.0      8000.0      8000.0      8000.0   

period                 2026-09-30  2026-10-31  2026-11-30  2026-12-31  
id item_sku renewable                                                  
1  1        True           8000.0      8000.0      8000.0      8000.0  
   2        True           8000.0      8000.0      8000.0      8000.0  

[2 rows x 36 columns]

$\begin{matrix}
 \text{Header} & \to \text{TCV 36k} & \to & \begin{matrix}12k & \to & \text{Year 1}\\ 12k & \to & \text{Year 2}\\ 12k & \to & \text{Year 3}\end{matrix}
 \\\\
\text{Line} & \to \text{TCV 36k} & \to & \begin{matrix}8k & \to & \text{Year 1}\\ 12k & \to & \text{Year 2}\\ 16k & \to & \text{Year 3}\end{matrix}
\end{matrix}$

<div class="alert alert-block alert-info">
Annualizing by the line start and end dates is preferred over annualizing by header start and end dates.
</div>